In [1]:
import pandas as pd
import numpy as np
import statistics
import collections
import statsmodels
from fbprophet import Prophet

import hts
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree

import pmdarima as pm


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
# import the data files
data = pd.read_excel('./data.xlsx',sheet_name='Quadro', skiprows=11, usecols="A:D", header=None, index_col=0)

In [3]:
first_row = data.loc["Years",:]
data.columns=first_row
data.drop(data.index[[0]],inplace=True)

In [4]:
# lowercase the column names
data.columns = [col_name.lower() for col_name in data.columns]

In [5]:
# cleanup region name 
data["nutsiii"] = data["nutsiii"].apply(lambda x: x.replace(" ", "_"))

# map the full NUTSII names to abbreviations
nutsii_dict = {
    "Norte": "NT",
    "Centro": "CT",
    "Área Metropolitana de Lisboa": "LIS",
    "Alentejo": "ALT",
    "Algarve": "ALG",
    "Região Autónoma dos Açores": "AÇR",
    "Região Autónoma da Madeira": "MAD",
}

data["nutsii"] = data["nutsii"].map(nutsii_dict)

In [6]:
data.reset_index(inplace=True)
data.columns.values[0]="date"
data

,date,nutsii,nutsiii,nº of nights
0,2009-12-31,NT,Alto_Minho,0
1,2009-12-31,NT,Cávado,515.714
2,2009-12-31,NT,Ave,295.402
3,2009-12-31,NT,Área_Metropolitana_do_Porto,0
4,2009-12-31,NT,Alto_Tâmega,0
...,...,...,...,...
403,2020-12-31,AÇR,Ilha_do_Faial,44.39
404,2020-12-31,AÇR,Ilha_das_Flores,22.416
405,2020-12-31,AÇR,Ilha_do_Corvo,1.166
406,2020-12-31,MAD,Ilha_da_Madeira,2282.241


## Ground Up Example
Make custom forecasts and reconciling them 

In [7]:
# specifying levels in the hierarchy
level_names = ['nutsii', 'nutsiii']
#specifying the levels to include in the hierarchy structure
hier = [['nutsii'], ['nutsiii']]
#get a wide pandas.DataFrame with the individual time series to create forecasts.
wide_df, sum_mat, sum_mat_labels = hts.functions.get_hierarchichal_df(data,
                                                                      level_names=level_names,
                                                                      hierarchy=hier,
                                                                      date_colname='date',
                                                                      val_colname='nº of nights')

In [8]:
wide_df = pd.DataFrame(wide_df,
                    index=pd.date_range(end='2021-12', freq='A', periods=12),
                    columns=wide_df.columns)
wide_df

nutsii_nutsiii,ALG_Algarve,ALT_Alentejo_Central,ALT_Alentejo_Litoral,ALT_Alto_Alentejo,ALT_Baixo_Alentejo,ALT_Lezíria_do_Tejo,AÇR_Ilha_Graciosa,AÇR_Ilha_Terceira,AÇR_Ilha_das_Flores,AÇR_Ilha_de_Santa_Maria,...,Ilha_de_São_Miguel,Ilha_Terceira,Ilha_Graciosa,Ilha_de_São_Jorge,Ilha_do_Pico,Ilha_do_Faial,Ilha_das_Flores,Ilha_do_Corvo,Ilha_da_Madeira,Ilha_de_Porto_Santo
2009-12-31,12927.603,383.064,277.538,175.475,172.589,95.649,9.512,141.563,0,22.164,...,0.000,141.563,9.512,0.000,0.000,79.862,0.000,0.000,0.000,240.426
2010-12-31,13247.45,403.992,307.195,183.086,179.06,99.225,16.043,131.507,14.229,23.933,...,0.000,131.507,16.043,0.000,0.000,81.032,14.229,0.000,4776.315,217.210
2011-12-31,13979.866,411.336,376.595,189.742,167.212,98.767,16.449,140.675,15.8,23.122,...,0.000,140.675,16.449,0.000,0.000,75.845,15.800,0.000,5289.188,276.098
2012-12-31,14326.774,389.28,370.033,150.747,142.039,90.046,14.072,135.388,13.666,21.098,...,0.000,135.388,14.072,0.000,0.000,73.615,13.666,0.000,5260.374,247.311
2013-12-31,14741.969,0,349.377,0,132.437,85.383,10.943,146.948,13.136,21.061,...,0.000,146.948,10.943,0.000,0.000,77.054,13.136,0.000,0.000,288.579
2014-12-31,16591.548,551.628,521.154,235.487,231.904,124.123,12.714,150.665,0,24.033,...,755.397,150.665,12.714,0.000,49.584,89.556,0.000,0.000,6164.884,341.982
2015-12-31,17256.396,646.812,585.969,274.683,278.892,137.952,13.035,162.693,0,23.12,...,931.702,162.693,13.035,0.000,57.564,101.300,0.000,0.000,6648.445,381.581
2016-12-31,19005.838,725.546,669.363,299.251,295.075,145.078,16.204,263.329,0,26.146,...,1081.578,263.329,16.204,0.000,66.628,115.291,0.000,0.000,7447.071,483.837
2017-12-31,20207.151,829.956,787.602,346.116,337.508,186.203,16.626,294.117,0,28.623,...,1281.791,294.117,16.626,0.000,71.633,115.658,0.000,0.000,7845.518,514.471
2018-12-31,20443.247,898.853,864.051,382.257,350.619,180.165,16.194,322.97,36.918,28.844,...,1414.615,322.970,16.194,0.000,106.745,158.933,36.918,0.000,7849.500,494.766


In [9]:
wide_df.index[0]

Timestamp('2009-12-31 00:00:00', freq='A-DEC')

In [10]:
# Create a DataFrame to store new forecasts in
# Here we just do an average
forecasts = pd.DataFrame(index=['forecast'], columns=wide_df.columns)

for col in wide_df.columns:
        forecasts[col] = statistics.mean(wide_df[col])
forecasts

nutsii_nutsiii,ALG_Algarve,ALT_Alentejo_Central,ALT_Alentejo_Litoral,ALT_Alto_Alentejo,ALT_Baixo_Alentejo,ALT_Lezíria_do_Tejo,AÇR_Ilha_Graciosa,AÇR_Ilha_Terceira,AÇR_Ilha_das_Flores,AÇR_Ilha_de_Santa_Maria,...,Ilha_de_São_Miguel,Ilha_Terceira,Ilha_Graciosa,Ilha_de_São_Jorge,Ilha_do_Pico,Ilha_do_Faial,Ilha_das_Flores,Ilha_do_Corvo,Ilha_da_Madeira,Ilha_de_Porto_Santo
forecast,15959.920667,554.711583,564.998833,246.579417,243.65075,130.409333,13.910917,193.425167,12.880917,24.039917,...,617.477167,193.425167,13.910917,1.416917,41.8935,98.084417,12.880917,0.097167,5046.729083,342.128333


In [11]:
#Store the forecasts in a dictionary to be passed to the reconciliation algorithm
pred_dict = collections.OrderedDict()
# Add predictions to dictionary in same order as summing matrix
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=forecasts[label].values, columns=['yhat'])

In [12]:
#Reconcile the forecasts.Here we use OLS optimal reconciliation. 
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})

#Then, put reconciled forecasts in the same wide DataFrame format.
revised_forecasts = pd.DataFrame(data=revised[0:,0:],
                                 index=forecasts.index,
                                 columns=sum_mat_labels)

In [13]:
revised_forecasts

,total,Ilha_de_Porto_Santo,Ilha_da_Madeira,Ilha_do_Corvo,Ilha_das_Flores,Ilha_do_Faial,Ilha_do_Pico,Ilha_de_São_Jorge,Ilha_Graciosa,Ilha_Terceira,...,AÇR_Ilha_de_São_Miguel,AÇR_Ilha_Terceira,AÇR_Ilha_Graciosa,AÇR_Ilha_de_São_Jorge,AÇR_Ilha_do_Pico,AÇR_Ilha_do_Faial,AÇR_Ilha_das_Flores,AÇR_Ilha_do_Corvo,MAD_Ilha_da_Madeira,MAD_Ilha_de_Porto_Santo
forecast,41885.808675,344.191664,5048.792414,0.847469,13.631219,98.834719,42.643802,2.167219,14.661219,194.175469,...,618.227469,194.175469,14.661219,2.167219,42.643802,98.834719,13.631219,0.847469,5048.792414,344.191664


## Reconcile Pre-Computed Forecasts Example

In [14]:
# create the bottom level data
data_bottom_level = data.pivot(index="date", columns="nutsii_nutsiii", values="nº of nights")

# create the middle level data
data_middle_level = data.groupby(["date", "nutsii"]).sum().reset_index(drop=False).pivot(index="date", columns="nutsii", values="nº of nights")

# create the total level data
data_total = data.groupby("date")["nº of nights"].sum().to_frame().rename(columns={"nº of nights": "total"})


In [15]:
# join the DataFrames
hierarchy_data = data_bottom_level.join(data_middle_level).join(data_total)
hierarchy_data.index = pd.to_datetime(hierarchy_data.index)

print(f"Number of time series at the bottom level: {data_bottom_level.shape[1]}")
print(f"Number of time series at the middle level: {data_middle_level.shape[1]}")

hierarchy_data

Number of time series at the bottom level: 34
Number of time series at the middle level: 7


,ALG_Algarve,ALT_Alentejo_Central,ALT_Alentejo_Litoral,ALT_Alto_Alentejo,ALT_Baixo_Alentejo,ALT_Lezíria_do_Tejo,AÇR_Ilha_Graciosa,AÇR_Ilha_Terceira,AÇR_Ilha_das_Flores,AÇR_Ilha_de_Santa_Maria,...,NT_Tâmega_e_Sousa,NT_Área_Metropolitana_do_Porto,ALG,ALT,AÇR,CT,LIS,MAD,NT,total
date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,12927.603,383.064,277.538,175.475,172.589,95.649,9.512,141.563,0,22.164,...,0,0,12927.603,1104.315,253.101,1254.849,0.000,240.426,1026.364,16806.658
2010-12-31,13247.45,403.992,307.195,183.086,179.06,99.225,16.043,131.507,14.229,23.933,...,0,0,13247.450,1172.558,266.744,1254.948,0.000,4993.525,976.605,21911.83
2011-12-31,13979.866,411.336,376.595,189.742,167.212,98.767,16.449,140.675,15.8,23.122,...,0,0,13979.866,1243.652,271.891,1422.553,0.000,5565.286,966.599,23449.847
2012-12-31,14326.774,389.28,370.033,150.747,142.039,90.046,14.072,135.388,13.666,21.098,...,0,0,14326.774,1142.145,257.839,1367.118,0.000,5507.685,966.026,23567.587
2013-12-31,14741.969,0,349.377,0,132.437,85.383,10.943,146.948,13.136,21.061,...,0,0,14741.969,567.197,269.142,688.869,10040.808,288.579,756.368,27352.932
2014-12-31,16591.548,551.628,521.154,235.487,231.904,124.123,12.714,150.665,0,24.033,...,176.195,4030.435,16591.548,1664.296,1081.949,4486.949,12279.422,6506.866,6061.742,48672.772
2015-12-31,17256.396,646.812,585.969,274.683,278.892,137.952,13.035,162.693,0,23.12,...,220.657,4580.506,17256.396,1924.308,1289.414,5058.446,13468.659,7030.026,7001.899,53029.148
2016-12-31,19005.838,725.546,669.363,299.251,295.075,145.078,16.204,263.329,0,26.146,...,257.909,5210.981,19005.838,2134.313,1569.176,5643.792,14800.346,7930.908,7989.922,59074.295
2017-12-31,20207.151,829.956,787.602,346.116,337.508,186.203,16.626,294.117,0,28.623,...,291.477,5883.971,20207.151,2487.385,1808.448,6764.282,16695.206,8359.989,9008.846,65331.307


In [16]:
pd.date_range(end='2021-12', freq='A', periods=12)

DatetimeIndex(['2009-12-31', '2010-12-31', '2011-12-31', '2012-12-31',
               '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31',
               '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')

In [17]:
hierarchy_data = pd.DataFrame(hierarchy_data,
                    index=pd.date_range(end='2021-12', freq='A', periods=12),
                    columns=hierarchy_data.columns)
hierarchy_data

,ALG_Algarve,ALT_Alentejo_Central,ALT_Alentejo_Litoral,ALT_Alto_Alentejo,ALT_Baixo_Alentejo,ALT_Lezíria_do_Tejo,AÇR_Ilha_Graciosa,AÇR_Ilha_Terceira,AÇR_Ilha_das_Flores,AÇR_Ilha_de_Santa_Maria,...,NT_Tâmega_e_Sousa,NT_Área_Metropolitana_do_Porto,ALG,ALT,AÇR,CT,LIS,MAD,NT,total
2009-12-31,12927.603,383.064,277.538,175.475,172.589,95.649,9.512,141.563,0,22.164,...,0,0,12927.603,1104.315,253.101,1254.849,0.000,240.426,1026.364,16806.658
2010-12-31,13247.45,403.992,307.195,183.086,179.06,99.225,16.043,131.507,14.229,23.933,...,0,0,13247.450,1172.558,266.744,1254.948,0.000,4993.525,976.605,21911.83
2011-12-31,13979.866,411.336,376.595,189.742,167.212,98.767,16.449,140.675,15.8,23.122,...,0,0,13979.866,1243.652,271.891,1422.553,0.000,5565.286,966.599,23449.847
2012-12-31,14326.774,389.28,370.033,150.747,142.039,90.046,14.072,135.388,13.666,21.098,...,0,0,14326.774,1142.145,257.839,1367.118,0.000,5507.685,966.026,23567.587
2013-12-31,14741.969,0,349.377,0,132.437,85.383,10.943,146.948,13.136,21.061,...,0,0,14741.969,567.197,269.142,688.869,10040.808,288.579,756.368,27352.932
2014-12-31,16591.548,551.628,521.154,235.487,231.904,124.123,12.714,150.665,0,24.033,...,176.195,4030.435,16591.548,1664.296,1081.949,4486.949,12279.422,6506.866,6061.742,48672.772
2015-12-31,17256.396,646.812,585.969,274.683,278.892,137.952,13.035,162.693,0,23.12,...,220.657,4580.506,17256.396,1924.308,1289.414,5058.446,13468.659,7030.026,7001.899,53029.148
2016-12-31,19005.838,725.546,669.363,299.251,295.075,145.078,16.204,263.329,0,26.146,...,257.909,5210.981,19005.838,2134.313,1569.176,5643.792,14800.346,7930.908,7989.922,59074.295
2017-12-31,20207.151,829.956,787.602,346.116,337.508,186.203,16.626,294.117,0,28.623,...,291.477,5883.971,20207.151,2487.385,1808.448,6764.282,16695.206,8359.989,9008.846,65331.307
2018-12-31,20443.247,898.853,864.051,382.257,350.619,180.165,16.194,322.97,36.918,28.844,...,338.937,6319.435,20443.247,2675.945,2085.219,6777.827,17516.975,8344.266,9778.017,67621.496


In [18]:
hierarchy_data.index[0]

Timestamp('2009-12-31 00:00:00', freq='A-DEC')

In [19]:
#Creating the hierarchy
nutsii = data["nutsii"].unique()
nutsiii = data["nutsii_nutsiii"].unique()

total = {'total': list(nutsii)}
nutsii = {k: [x for x in nutsiii if x.startswith(k)] for k in nutsii}
hierarchy = {**total, **nutsii}

In [20]:
Htree = HierarchyTree.from_nodes(nodes=hierarchy, df=hierarchy_data)
Htree

- total
   |- NT
   |  |- NT_Alto_Minho
   |  |- NT_Cávado
   |  |- NT_Ave
   |  |- NT_Área_Metropolitana_do_Porto
   |  |- NT_Alto_Tâmega
   |  |- NT_Tâmega_e_Sousa
   |  |- NT_Douro
   |  - NT_Terras_de_Trás-os-Montes
   |- CT
   |  |- CT_Oeste
   |  |- CT_Região_de_Aveiro
   |  |- CT_Região_de_Coimbra
   |  |- CT_Região_de_Leiria
   |  |- CT_Viseu_Dão_Lafões
   |  |- CT_Beira_Baixa
   |  |- CT_Médio_Tejo
   |  - CT_Beiras_e_Serra_da_Estrela
   |- LIS
   |  - LIS_Área_Metropolitana_de_Lisboa
   |- ALT
   |  |- ALT_Alentejo_Litoral
   |  |- ALT_Baixo_Alentejo
   |  |- ALT_Lezíria_do_Tejo
   |  |- ALT_Alto_Alentejo
   |  - ALT_Alentejo_Central
   |- ALG
   |  - ALG_Algarve
   |- AÇR
   |  |- AÇR_Ilha_de_Santa_Maria
   |  |- AÇR_Ilha_de_São_Miguel
   |  |- AÇR_Ilha_Terceira
   |  |- AÇR_Ilha_Graciosa
   |  |- AÇR_Ilha_de_São_Jorge
   |  |- AÇR_Ilha_do_Pico
   |  |- AÇR_Ilha_do_Faial
   |  |- AÇR_Ilha_das_Flores
   |  - AÇR_Ilha_do_Corvo
   - MAD
      |- MAD_Ilha_da_Madeira
      - MAD_

In [21]:
sum_mat2, sum_mat_labels2 = hts.functions.to_sum_mat(Htree)

In [22]:
forecasts2 = pd.DataFrame(columns=hierarchy_data.columns, index=['forecast'])

In [23]:
 # Make forecasts made outside of hts package.
for col in hierarchy_data.columns:
    model = statsmodels.tsa.holtwinters.SimpleExpSmoothing(hierarchy_data[col].values).fit()
    fcst = list(model.forecast(1))
    forecasts2[col] = fcst

In [24]:
pred_dict2 = collections.OrderedDict()

In [25]:
# Add predictions to dictionary is same order as summing matrix
for label in sum_mat_labels2:
    pred_dict2[label] = pd.DataFrame(data=forecasts2[label].values, columns=['yhat'])

In [26]:
revised2 = hts.functions.optimal_combination(pred_dict2, sum_mat2, method='OLS', mse={})

In [27]:
# Put reconciled forecasts in nice DataFrame form
revised_forecasts2 = pd.DataFrame(data=revised2[0:,0:],
                                 index=forecasts2.index,
                                 columns=sum_mat_labels2)


In [28]:
revised_forecasts2

,total,MAD,AÇR,ALG,ALT,LIS,CT,NT,NT_Alto_Minho,NT_Cávado,...,AÇR_Ilha_de_São_Miguel,AÇR_Ilha_Terceira,AÇR_Ilha_Graciosa,AÇR_Ilha_de_São_Jorge,AÇR_Ilha_do_Pico,AÇR_Ilha_do_Faial,AÇR_Ilha_das_Flores,AÇR_Ilha_do_Corvo,MAD_Ilha_da_Madeira,MAD_Ilha_de_Porto_Santo
forecast,26461.383482,4606.500589,269.862966,7669.923354,1657.21347,5100.461544,3129.570512,4027.851047,389.962932,418.295648,...,348.580982,65.623858,-35.270229,-47.764228,-9.227144,48.909945,-26.765145,-49.083845,4106.441893,500.058697


Using prophet:

In [29]:
reg = HTSRegressor(model='prophet', revision_method='OLS')
reg = reg.fit(df=hierarchy_data, nodes=hierarchy)
preds_prophet = reg.predict(steps_ahead=0)

Fitting models: 100%|██████████████████████████████████████████████████████████████████| 14/14 [00:16<00:00,  1.16s/it]


In [30]:
preds_prophet

,total,NT,CT,LIS,ALT,ALG,AÇR,MAD,NT_Alto_Minho,NT_Cávado,...,AÇR_Ilha_de_São_Miguel,AÇR_Ilha_Terceira,AÇR_Ilha_Graciosa,AÇR_Ilha_de_São_Jorge,AÇR_Ilha_do_Pico,AÇR_Ilha_do_Faial,AÇR_Ilha_das_Flores,AÇR_Ilha_do_Corvo,MAD_Ilha_da_Madeira,MAD_Ilha_de_Porto_Santo
2009-12-31,2153.185282,1435.710839,-706.582045,-1276.892301,-1050.050340,5299.190597,-1568.054072,19.862604,-2886.085178,14437.403174,...,-1539.509195,11726.029286,-1516.795235,-1746.465834,-1555.775228,-1473.034096,-1537.206571,-1538.352924,1921.617402,-485.906563
2010-12-31,3719.108802,1869.494361,-1479.614515,-253.636443,-1707.161673,7123.823893,-2436.331692,602.534871,-3288.066764,22443.395404,...,-1845.376374,13051.543937,-1819.628537,-1757.460439,-1839.921193,-1759.469370,-1836.874356,-1843.335801,2710.020635,-840.526274
2011-12-31,5249.337896,2325.245123,-2241.736087,744.011352,-2346.609365,8882.260629,-3285.398463,1171.564707,-3668.092362,30230.807703,...,-2140.212787,14305.610126,-2111.636843,-1761.984129,-2113.497114,-2035.428329,-2125.545447,-2137.324182,3512.312827,-1187.067705
2012-12-31,3240.719936,1195.025549,-1252.092654,-112.186880,-1404.315823,6364.052870,-2109.064865,559.301739,-2974.826377,19723.454933,...,-1607.256861,11480.669853,-1588.981206,-1570.420453,-1608.794581,-1544.057697,-1606.300817,-1609.336160,1920.932060,-725.906511
2013-12-31,4841.250114,1606.362519,-2035.499147,936.106463,-2078.604870,8253.564195,-2995.931076,1155.252028,-3398.430968,27942.391679,...,-1923.899855,12876.373042,-1902.388607,-1587.824737,-1903.270877,-1840.732906,-1916.716412,-1925.060362,2694.718815,-1088.356296
2014-12-31,6407.173635,2040.146042,-2808.531617,1959.362321,-2735.716203,10078.197492,-3864.208696,1737.924296,-3800.412555,35948.383914,...,-2229.767034,14201.887695,-2205.221909,-1598.819341,-2187.416842,-2127.168180,-2216.384198,-2230.043239,3483.122049,-1442.976007
2015-12-31,7937.402695,2495.896827,-3570.653158,2957.010133,-3375.163866,11836.634245,-4713.275649,2306.954163,-4180.438150,43735.796219,...,-2524.603444,15455.953889,-2497.230212,-1603.343029,-2460.992759,-2403.127135,-2505.055286,-2524.031617,4285.414253,-1789.517426
2016-12-31,5928.784701,1365.677276,-2581.009694,2100.811918,-2432.870296,9318.426503,-3536.942234,1694.691226,-3487.172162,33228.443457,...,-1991.647515,12631.013618,-1974.574571,-1411.779349,-1956.290223,-1911.756501,-1985.810652,-1996.043591,2694.033497,-1328.356221
2017-12-31,7529.314844,1777.014269,-3364.416155,3149.105279,-3107.159314,11207.937845,-4423.808626,2290.641546,-3910.776749,41447.380205,...,-2308.290505,14026.716810,-2287.981969,-1429.183629,-2250.766515,-2208.431706,-2296.226244,-2311.767789,3467.820263,-1690.805994
2018-12-31,9095.238330,2210.797815,-4137.448594,4172.361153,-3764.270618,13032.571158,-5292.086427,2873.313844,-4312.758333,49453.372444,...,-2614.157680,15352.231465,-2590.815267,-1440.178230,-2534.912477,-2494.866976,-2595.894026,-2616.750663,4256.223509,-2045.425694


Using auto_arima:

In [31]:
reg = HTSRegressor(model='auto_arima', revision_method='OLS')
reg = reg.fit(df=hierarchy_data, nodes=hierarchy)
preds_autoarima = reg.predict(steps_ahead=0)

Fitting models: 100%|██████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  3.23it/s]


In [32]:
preds_autoarima

,total,NT,CT,LIS,ALT,ALG,AÇR,MAD,NT_Alto_Minho,NT_Cávado,...,AÇR_Ilha_de_São_Miguel,AÇR_Ilha_Terceira,AÇR_Ilha_Graciosa,AÇR_Ilha_de_São_Jorge,AÇR_Ilha_do_Pico,AÇR_Ilha_do_Faial,AÇR_Ilha_das_Flores,AÇR_Ilha_do_Corvo,MAD_Ilha_da_Madeira,MAD_Ilha_de_Porto_Santo
2009-12-31,8190.044883,-2533.956415,7079.983612,23697.274412,-6546.773105,-4095.022441,-7094.963155,-2316.498025,-165.854802,-484.689442,...,1235.345273,866.768900,703.803088,689.892172,689.892172,787.976588,702.481158,689.892172,1092.823848,-3626.780263
2010-12-31,7241.566227,-2175.629933,7889.006899,17235.851175,-5066.806063,-3054.380451,-6071.918372,-1514.557028,-290.456608,-338.151782,...,958.765657,982.377797,843.258457,829.347540,829.347540,927.431957,835.742955,829.347540,1293.333271,-3468.963205
2011-12-31,7495.943020,-2315.899278,7667.715916,18660.408198,-5305.862335,-3181.524161,-6288.877206,-1740.018114,-290.273057,-367.067489,...,994.549759,945.625061,810.172648,796.261731,796.261731,894.346148,813.935461,796.261731,1223.701158,-3539.600436
2012-12-31,7595.041806,-2338.279837,7586.471714,19078.347626,-5385.791381,-3155.421489,-6365.872396,-1824.412432,-275.887445,-381.771958,...,1045.538823,944.812973,792.318064,778.407147,778.407147,876.491564,789.641787,778.407147,1193.316321,-3531.596158
2013-12-31,7584.210700,-2392.400844,7602.213298,19119.550907,-5387.088642,-3175.027697,-6368.628373,-1814.407948,-245.985511,-380.945335,...,1109.431779,927.824286,787.407586,773.496669,773.496669,871.581086,787.941966,773.496669,1192.114282,-3584.515126
2014-12-31,15635.690403,-2199.218957,8176.251796,20584.459367,-5067.799140,1173.233703,-5799.789858,-1231.446508,109.538396,-335.243181,...,1232.806175,999.000700,842.032682,828.121765,828.121765,926.206182,839.679081,828.121765,1252.890813,-3452.109770
2015-12-31,19702.468155,-3148.368426,6328.742516,25998.980248,-2823.000870,2789.440301,-6778.464536,-2664.861078,-531.884824,-878.066554,...,1627.074138,711.449238,558.015865,544.104948,586.250954,642.189365,547.408197,544.104948,785.515668,-3933.884094
2016-12-31,21087.343330,-3258.564052,6013.776389,27144.727058,-2400.420174,3383.039355,-6892.286188,-2902.929058,-596.086534,-961.484360,...,1622.299688,680.684074,516.186188,502.275271,551.204214,600.359688,512.176806,502.275271,708.370540,-3966.934592
2017-12-31,22681.657767,-3446.620340,5625.044372,28760.638626,-2020.874791,4001.315923,-7022.707043,-3215.138981,-694.197219,-1087.191303,...,1560.841231,731.639931,461.829245,447.918329,504.551599,546.002745,452.384718,447.918329,590.202350,-4036.822689
2018-12-31,24779.375415,-3604.335080,5210.104188,30432.673872,-1616.462639,5096.294935,-7196.406056,-3542.493805,-776.048574,-1204.926544,...,1527.320518,654.403607,417.132734,403.221817,464.109298,501.306234,412.126430,403.221817,518.384517,-4122.719596
